<a href="https://colab.research.google.com/github/marcduda/various_projects/blob/master/quotes_generation_for_git.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install the libraries for parsing
!pip3 install pandas
!pip3 install selenium
!pip3 install webdriver-manager
!pip3 install langdetect

     |████████████████████████████████| 911kB 16.8MB/s 


In [ ]:
# launch the parsing with a chrome browser and save the content in a file
import random
import unicodedata
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from time import sleep, strftime
from random import randint
import pandas as pd
webdriver = webdriver.Chrome(ChromeDriverManager().install())

total_list_quotes = []

for i in range(1,101):
    print(i)
    webdriver.get('https://www.goodreads.com/quotes/tag/inspirational?page='+str(i))
    list_quotes = webdriver.find_elements_by_css_selector(".quoteText")
    list_quotes = [l.text for l in list_quotes]
    list_quotes = [l.split('”\n―')[0].replace('<br>', '').replace('\n', ' ') for l in list_quotes]
    total_list_quotes += list_quotes
    sleep(random.randint(20, 60))

quotes_string =''
for l in total_list_quotes:
    type_string = str(l) + '\n'
    quotes_string+=type_string

text_file = open("/content/quotes.txt", "w")
text_file.write(quotes_string)
text_file.close()


In [ ]:
# filter the quotes to get only those not too short or not too long and in english only, save again in file
from langdetect import detect
with open('/content/quotes.txt') as file:
    file_contents = file.read()
contents = file_contents.split('\n')
total_list = [q.replace("”", "").replace("“", "") for q in contents 
              if (len(q.split(" "))<=30)&(len(q.split(" "))>=5)] 
total_list = [q for q in total_list if (detect(q)=="en")]
total_list = list(set(total_list))
              
total_list[0]
quotes_string =''
for l in total_list:
    type_string = str(l) + '\n'
    quotes_string+=type_string

text_file = open("/content/short_english_quotes.txt", "w")
text_file.write(quotes_string)
text_file.close()

In [ ]:
# user tensorflow V1 for the NLP library, load the gpt_2_simple library
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
# launch the finetuning
# NB : if this cell fails, the kernel needs to be reloaded before relaunching this cell
file_name = '/content/short_english_quotes.txt'
sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset=file_name,
              model_name='/content/124M/124M',
              steps=1000,
              restore_from='fresh',
              run_name='run2',
              print_every=10,
              sample_every=100,
              save_every=100
              )

In [ ]:
# mount drive
gpt2.mount_gdrive()

Mounted at /content/drive


In [ ]:
# save checkpoints to drive
!cp -r checkpoint/ drive/MyDrive